In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ngmix
import numpy as np
import galsim

In [26]:
from prepsf_admom import PrePSFAdmom
from ngmix.prepsfmom import PGaussMom

In [98]:
from ngmix import Jacobian, Observation

rng = np.random.RandomState(seed=100)
image_size = 101
psf_image_size = 53
pixel_scale = 0.2
snr = 2e1
fwhm = 1e-5
psf_fwhm = 1
mom_fwhm = 2

cen = (image_size - 1)/2
psf_cen = (psf_image_size - 1)/2
gs_wcs = galsim.ShearWCS(
    pixel_scale, galsim.Shear(g1=-0.1, g2=0.06)).jacobian()
scale = np.sqrt(gs_wcs.pixelArea())
shift = rng.uniform(low=-scale/2, high=scale/2, size=2)
psf_shift = rng.uniform(low=-scale/2, high=scale/2, size=2)
xy = gs_wcs.toImage(galsim.PositionD(shift))
psf_xy = gs_wcs.toImage(galsim.PositionD(psf_shift))

jac = Jacobian(
    y=cen + xy.y, x=cen + xy.x,
    dudx=gs_wcs.dudx, dudy=gs_wcs.dudy,
    dvdx=gs_wcs.dvdx, dvdy=gs_wcs.dvdy)

psf_jac = Jacobian(
    y=psf_cen + psf_xy.y, x=psf_cen + psf_xy.x,
    dudx=gs_wcs.dudx, dudy=gs_wcs.dudy,
    dvdx=gs_wcs.dvdx, dvdy=gs_wcs.dvdy)

gal = galsim.Gaussian(
    fwhm=fwhm
).shear(
    e1=-0.1, e2=0.2
).withFlux(
    400
).shift(
    dx=shift[0], dy=shift[1]
)

gal = galsim.Exponential(
    half_light_radius=fwhm,
).shear(
    e1=-0.1, e2=0.2
).withFlux(
    400
).shift(
    dx=shift[0], dy=shift[1]
)


psf = galsim.Gaussian(
    fwhm=psf_fwhm
).shear(
    g1=0.3, g2=-0.15
)
im = galsim.Convolve([gal, psf]).drawImage(
    nx=image_size,
    ny=image_size,
    wcs=gs_wcs
).array
noise = np.sqrt(np.sum(im**2)) / snr
wgt = np.ones_like(im) / noise**2

psf_im = psf.shift(
    dx=psf_shift[0], dy=psf_shift[1]
).drawImage(
    nx=psf_image_size,
    ny=psf_image_size,
    wcs=gs_wcs
).array

im_true = gal.drawImage(
    nx=image_size,
    ny=image_size,
    wcs=gs_wcs,
    method='no_pixel').array
obs_true = Observation(
    image=im_true,
    jacobian=jac,
)

obs = Observation(
    image=im + rng.normal(size=im.shape, scale=noise),
    weight=wgt,
    jacobian=jac,
    psf=Observation(image=psf_im, jacobian=psf_jac),
)

In [99]:
res = PrePSFAdmom(min_fwhm=2.0, delta_fwhm=0.1).go(obs, 3)
PGaussMom(fwhm=2).go(obs)["s2n"], res["s2n"], res["flagstr"], res

(17.702765300829302,
 17.702765300829302,
 'T <= 0',
 {'flags': 8,
  'flagstr': 'T <= 0',
  'flux': 433.096182660285,
  'mom': array([         nan,          nan, -49.66334165,  25.57519569,
         -22.37381732, 433.09618266]),
  'mom_cov': array([[   1.        ,    0.        ,    0.        ,    0.        ,
             0.        ,    0.        ],
         [   0.        ,    1.        ,    0.        ,    0.        ,
             0.        ,    0.        ],
         [   0.        ,    0.        ,  872.91425126,  -73.40553645,
           341.80395948, -171.11304099],
         [   0.        ,    0.        ,  -73.40553645,  762.99183988,
          -171.08317537,   85.64887457],
         [   0.        ,    0.        ,  341.80395948, -171.08317537,
           900.78752488,  176.97601242],
         [   0.        ,    0.        , -171.11304099,   85.64887457,
           176.97601242,  598.53077881]]),
  'flux_flags': 0,
  'flux_flagstr': '',
  'T_flags': 0,
  'T_flagstr': '',
  'flux_err': 24